In [2]:
import numpy as np

In [15]:
def lu_decomposition(A, dU=None):
    n = A.shape[0]
    
    if dU is None:
        dU = np.ones(n)
    
    L = np.zeros((n, n))
    U = np.zeros((n, n))
    
    for i in range(n):
        U[i, i] = dU[i]
    
    L[0, 0] = A[0, 0] / U[0, 0]
    
    for j in range(1, n):
        U[0, j] = A[0, j] / L[0, 0]
    
    for i in range(1, n):
        L[i, 0] = A[i, 0] / U[0, 0]
    
    for p in range(1, n):
        sum_l = 0
        for k in range(p):
            sum_l += L[p, k] * U[k, p]
        L[p, p] = (A[p, p] - sum_l) / U[p, p]  
        # L[1,1] = (7-1*1)/3 = 2
        # L[2,2] = (12.5-3*0)/4 = 3.125
        
        for j in range(p+1, n):
            sum_u = 0
            for k in range(p):
                sum_u += L[p, k] * U[k, j]
            U[p, j] = (A[p, j] - sum_u) / L[p, p]
    
        for i in range(p+1, n):
            sum_l = 0
            for k in range(p):
                sum_l += L[i, k] * U[k, p]
            L[i, p] = (A[i, p] - sum_l) / U[p, p]
    
    return L, U


# After the decomposition, A = LU and det(A) = det(L) * det(U) 
# Therefore det(A) = det(L) * det(U) = product of the diagonal elements of U and L
def find_determinant(A, dU):

    return np.prod([A[i][i] for i in range(A.shape[0])]) * np.prod([dU[i] for i in range(len(dU))])


# Given A = LU we have to solve the system
# Ax = b.
# Since A = LU we can solve for
# LUx = b

# Using substitution we can substitute Ux as y , therefore we will need to solve 2 triangular systems

# Ly = b
# and 
# Ux = y
def solve_system(A, b):
    L, U = lu_decomposition(A)
    n = A.shape[0]
    
    # Ly = b (fofward substitution)
    y = np.zeros(n)
    for i in range(n):
        y[i] = b[i]
        for j in range(i):
            y[i] -= L[i, j] * y[j]
        y[i] /= L[i, i]
    
    # Ux = y (backward substitution)
    x = np.zeros(n)
    for i in range(n-1, -1, -1):
        x[i] = y[i]
        for j in range(i+1, n):
            x[i] -= U[i, j] * x[j]
        x[i] /= U[i, i]
    
    return x

# Verify the solution using the norm of the difference between the product of A and x minus b
def verify_solution(A, x, b):
    return (np.sqrt(np.sum((np.dot(A, x) - b)**2)) < 1e-9)

# Solve using numpy
def solve_system_numpy(A, b):
    return np.linalg.solve(A, b)


In [ ]:
import numpy as np

n = 3

A = np.array([
    [4, 2, 3],
    [2, 7, 5.5],
    [6, 3, 12.5]
])
Ainit = A.copy()

b = [21.6, 33.6, 51.6]

dU = [2, 3, 4]

valid = True

for k in range(1, n+1):
    Ak = A[:k, :k]
    det = np.linalg.det(Ak)
    if abs(det) < 1e-8:
        valid = False
        print("Nu se poate calcula o descompunere LU")
        break

if valid:
    L, U = lu_decomposition(A, dU)
    print("L =\n", L)
    print("U =\n", U)
    
    x = solve_system(Ainit, b)
    xNumPy = solve_system_numpy(Ainit, b)
    print("Determinant using LU decomposition:\n", find_determinant(L, dU))
    print("Solution to the system Ax = b\n x =", x)
    print("The solution is valid:", verify_solution(Ainit, x, b))
    print("Solution to the system Ax = b using solely numpy\n x =", xNumPy)

    


L =
 [[2. 0. 0.]
 [1. 2. 0.]
 [3. 0. 2.]]
U =
 [[2.  1.  1.5]
 [0.  3.  2. ]
 [0.  0.  4. ]]
Determinant using LU decomposition:
 192.0
Solution to the system Ax = b
 x = [2.5 2.2 2.4]
The solution is valid: True
Solution to the system Ax = b using numpy
 x = [2.5 2.2 2.4]
